## P1

In [1]:
sum = 0

for x in range(1,1000):
    if x % 3 == 0 or x % 5 ==0:
        sum+=x

## P2

In [3]:
import numpy as np
def gen_Fibonacci(upper_limit):
    nums = []
    nums.append(1)
    nums.append(2)
    while nums[-2]+nums[-1] <= upper_limit:
        nums.append(nums[-2]+nums[-1])
    return nums

nums = gen_Fibonacci(4e6)

sum = 0
for x in nums:
    if x % 2 == 0:
#         print(x)
        sum += x
sum

4613732

## P3

In [4]:
def ck_prime(num):
    x = 2
    while num % x != 0:
        x+=1
#     is_prime = any((num % np.array(range(2,num)))==0)==False ## Vectorized version (memory issue)
    return(x==num)

def find_max_prime(num):
    for x in range(2,600851475143+1):
#         if x % 10000 == 0:
#             print(x)
        if num % x == 0:
            if ck_prime(num / x):
                return(int(num / x))
        
find_max_prime(600851475143)

6857

## P64

In [5]:
import numpy as np
def next_set(N,b,c):
    """
    Helper function to get next set based on N, bn and cn
    For the special case of initialization, set b0 = np.floor(np.sqrt(N)) and c0 = 1
    """
    rt = np.floor(np.sqrt(N))
    c2 = (N - b**2)/c
    a2 = np.floor((b+rt)/c2)
    b2 = a2*c2 -b
    assert(c2.is_integer())
    return(int(a2),int(b2),int(c2))

def get_period(N):
    if np.sqrt(N).is_integer():
        return(0)
    else:
        a1,b1,c1 = next_set(N,np.floor(np.sqrt(N)),1)
        i=1
        an,bn,cn = next_set(N,b1,c1)
        while (an,bn,cn) != (a1,b1,c1):
            an,bn,cn = next_set(N,bn,cn)
            i+=1
        return(i)
    
store = []
for N in range(1,10001):
    if get_period(N) % 2 == 1:
        store.append(N)
print(len(store))

1322


## P252

In [1]:
import numpy as np
def rand_gen(N):
    """Generate randome numbers"""
    T = []
    Sn = 290797
    for n in range(N):
        Sn = Sn**2 % 50515093
        T.append((Sn % 2000) - 1000)
    return T
randoms = np.array(rand_gen(1000))
dots = randoms.reshape([-1,2])
print(dots[:2])

[[ 527  144]
 [-488  732]]


In [109]:
def get_vector(dot1,dot2):
    """Get the vector between dot2 and dot1"""
    x = dot2[0]-dot1[0]
    y = dot2[1]-dot1[1]
    return([x,y])
    
def get_angle_area(v1,v2):
    """Get the angle between vector v2 and v1, ranging from [-180,180]"""
    v1_mag = np.linalg.norm(v1)
    v2_mag = np.linalg.norm(v2)
    
    cos_theta = np.dot(v1,v2)/v1_mag/v2_mag
    sin_theta = np.cross(v1,v2)/v1_mag/v2_mag
    if sin_theta >= 0:
        angle = np.rad2deg(np.arccos(cos_theta))
    else:
        angle = - np.rad2deg(np.arccos(cos_theta))
    area = np.cross(v1,v2).tolist()/2
    return(angle,area)

maxArea = 0

def max_area(acc_area):
    global maxArea
    maxArea = max(acc_area,maxArea)

In [110]:
def ck_convex(v1,v2,v3,v4):
    """check if the object is convex when adding a new dot"""
    convex = (get_angle_area(v1,v2)[0] >= 0) and (get_angle_area(v2,v3)[0] >= 0) and (get_angle_area(v3,v4)[0] >= 0)
    return(convex)

def ck_exclusive(v1,v2,v3,v4,v2_,v3_):
    """check if the dot falls outside the new triangle"""
    inclusive = ck_convex(v1,v2_,v3_,v4) and (get_angle_area(v2,v2_)[0] >= 0) and (get_angle_area(v3,v3_)[0] <= 0)
    return(not inclusive)
    
def add_one_dot(curr_list,new_dot,other_dots,acc_area):
    """add one dot to curr from candi and add the area of the new triangle into acc_ares"""
    v1 = get_vector(curr_list[-2],curr_list[-1])
    v2 = get_vector(curr_list[-1],new_dot)
    v3 = get_vector(new_dot,curr_list[0])
    v4 = get_vector(curr_list[0],curr_list[1])
    convex = ck_convex(v1,v2,v3,v4)
    exclusive = True
    if convex and other_dots != None:
        for x in other_dots:
            v2_ = get_vector(curr_list[-1],x)
            v3_ = get_vector(x,curr_list[0])
            exclusive = ck_exclusive(v1,v2,v3,v4,v2_,v3_)
            if not exclusive:
                break
    added = (convex and exclusive)
    if added:
        acc_area += get_angle_area(v2,v3)[1]
    return(added, acc_area)

def expand_convex_hole(curr_list,candi_dots,acc_area):
    if candi_dots != None:
        for x in candi_dots:
            other_dots = candi_dots.copy()
            other_dots.remove(x)
            added,acc_area2 = add_one_dot(curr_list,x,other_dots,acc_area)
            if added:
                next_list = curr_list.copy()
                next_list.append(x) 
                max_area(acc_area2)
                expand_convex_hole(next_list,other_dots,acc_area2)  

In [111]:
maxArea = 0
# [[0.5,1.51]]

# expand_convex_hole([[1,0],[1,1],[0.9,1]],[[2,0],[0,-6]],0.05)
expand_convex_hole([[1,0],[1,1]],[[0,2],[0,-1]],0)

maxArea

0.8499999999999999

In [ ]:
maxArea = 0

dots_list = dots.tolist()
for i,x in enumerate(dots_list):
    dots2 = dots_list.copy()
    dots2.remove(x)
    for j,y in enumerate(dots2):
        dots3 = dots2.copy()
        dots3.remove(y) 
        expand_convex_hole([x,y],dots3,0)
        print(i,j,maxArea)
        
maxArea

[[527, 144], [-488, 732]]
13658.0
[[527, 144], [-454, -947]]
13658.0
[[527, 144], [-883, -38]]
34649.0
[[527, 144], [-754, 70]]
34649.0
[[527, 144], [-178, 540]]
34649.0
[[527, 144], [54, 316]]
52042.0
[[527, 144], [-389, 938]]
52042.0
[[527, 144], [913, -860]]
52042.0
[[527, 144], [806, 528]]
52042.0
[[527, 144], [331, 698]]
52042.0
[[527, 144], [-60, 138]]
52730.5
[[527, 144], [-811, 286]]
52730.5
[[527, 144], [-646, -422]]
52730.5
[[527, 144], [-665, -726]]
52730.5
[[527, 144], [491, 368]]
52730.5
[[527, 144], [-827, -607]]
52730.5
[[527, 144], [-893, -826]]
52730.5
[[527, 144], [-477, -23]]
52730.5
[[527, 144], [273, -175]]
52730.5
[[527, 144], [-212, 817]]
52730.5
[[527, 144], [-278, 293]]
52730.5
[[527, 144], [-188, 969]]
52730.5
[[527, 144], [-922, 255]]
52730.5
[[527, 144], [-373, -412]]
61617.0
[[527, 144], [708, 670]]
61617.0
[[527, 144], [-526, -246]]
61617.0
[[527, 144], [343, -756]]
61617.0
[[527, 144], [-370, -526]]
61617.0
[[527, 144], [-930, 111]]
61617.0
[[527, 144], [

/sasgrid01/datascience/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in arccos
  from ipykernel import kernelapp as app


73437.0
[[527, 144], [546, 680]]
73437.0
[[527, 144], [530, 381]]
73437.0
[[527, 144], [-304, 127]]
73437.0
[[527, 144], [348, -318]]
73437.0
[[527, 144], [-779, 757]]
73437.0
[[527, 144], [56, -429]]
73437.0
[[527, 144], [-319, -349]]
73899.0
[[527, 144], [92, 155]]
73899.0
[[527, 144], [-87, 386]]
73899.0
[[527, 144], [514, 463]]
73899.0
[[527, 144], [699, 72]]
73899.0
[[527, 144], [120, 410]]
73899.0
[[527, 144], [144, 655]]
73899.0
[[527, 144], [-808, 222]]
73899.0
[[527, 144], [327, 394]]
73899.0
[[527, 144], [863, -835]]
73899.0
[[527, 144], [55, -9]]
73899.0
[[527, 144], [-921, -333]]
73899.0
[[527, 144], [149, -331]]
73899.0
[[527, 144], [-313, 997]]
73899.0
[[527, 144], [528, -412]]
73899.0
[[527, 144], [694, -315]]
73899.0
[[527, 144], [-69, -499]]
73899.0
[[527, 144], [752, 85]]
73899.0
[[527, 144], [-272, 270]]
73899.0
[[527, 144], [700, -966]]
73899.0
[[527, 144], [748, 876]]
73899.0
[[527, 144], [283, -331]]
73899.0
[[527, 144], [944, 225]]
73899.0
[[527, 144], [129, 498]

In [296]:
# 73899.0

0